# Comparison between ClimateTRACE and C40 inventories

This analysis compares city aggregated estimates from [climateTRACE](https://climatetrace.org/) to estimates [GPC](https://ghgprotocol.org/ghg-protocol-cities) compliant C40 city inventories downloaded from [here](https://www.c40knowledgehub.org/s/article/C40-cities-greenhouse-gas-emissions-interactive-dashboard?language=en_US).

I am not sure if the C40 inventories are high quality. Comparing to downscaled observations would not be a fair comparison. 

In [1]:
import json
import os
import fnmatch
import pandas as pd
import tarfile
import os
import requests
from sqlalchemy import create_engine, MetaData, text
from sqlalchemy.orm import sessionmaker
from tqdm import tqdm

In [2]:
from utils import (
    get_c40_data, 
    filter_out_notation_keys,
    climatetrace_file_names,
    load_climatetrace_file,
    point_to_lat_lon,
    lat_lon_to_locode
)

## Read raw C40 data

**Units**: metric tonnes CO2-eq. (I am assuming these are units since they should be following the GPC)

In [3]:
df_c40_raw = get_c40_data()

### filter C40

In [5]:
refnos = ['II.4.1']
columns = ['city', 'locode', 'year'] + refnos

df_tmp = filter_out_notation_keys(df_c40_raw, refnos)
df_c40 = (
    df_tmp
    .loc[:, columns]
    .rename(columns = {'II.4.1': 'emissions_c40'})
)

### Read ClimateTRACE

**Units**: Units are tonnes 

In [6]:
asset_file = './transportation/asset_domestic-aviation_emissions.csv'
df_ct_raw = load_climatetrace_file(asset_file)
filt = (df_ct_raw['gas'] == 'co2e_100yr')
df_tmp = df_ct_raw.loc[filt]

In [7]:
points = set(df_tmp['st_astext'])
df_points = pd.DataFrame([point_to_lat_lon(point) for point in points])

df_merged = df_tmp.merge(df_points, on='st_astext')
df_unique = df_merged[['lon','lat']].drop_duplicates()
print(f"number unique assets: {len(df_unique)}")

db_uri = "postgresql://ccglobal:@localhost/ccglobal"
engine = create_engine(db_uri)
metadata_obj = MetaData()
Session = sessionmaker(bind=engine)
session = Session()

output_list = []
for _, row in tqdm(df_unique.iterrows()):
    lat = row['lat']
    lon = row['lon']
    locode = lat_lon_to_locode(session, lat, lon)
    output_list.append({'lon':lon, 'lat': lat, 'locode': locode})

session.close()

df_locodes = pd.DataFrame(output_list)
df_merged_locodes = df_merged.merge(df_locodes, on =['lat','lon'])
filt = df_merged_locodes['locode'].notnull()
df_data = df_merged_locodes[filt]

number unique assets: 4815


4815it [00:34, 138.01it/s]


In [8]:
df_ct = (
    df_data
    .assign(year = lambda row: pd.to_datetime(row['start_time']).dt.year)
    .loc[:, ['locode', 'year', 'emissions_quantity', 'emissions_factor_units']]
    .rename(columns = {'emissions_quantity': 'emissions_ct'})
)

In [9]:
# check the units
set(df_ct['emissions_factor_units'])

{'tonnes_gas_per_tonnes_fuel'}

### Comparison

In [10]:
df_int = pd.merge(df_ct, df_c40, on = ['year', 'locode'], how='inner')
df_int['diff'] = df_int['emissions_ct'] - df_int['emissions_c40']
df_int['percent_error'] = (df_int['diff'] / df_int['emissions_c40']) * 100

In [12]:
df_int.loc[df_int['diff'].notnull()]

,locode,year,emissions_ct,emissions_factor_units,city,emissions_c40,diff,percent_error
0,US PHX,2016,3.340930e+06,tonnes_gas_per_tonnes_fuel,Phoenix,720710.439481,2620219.262927,363.560609
1,US PHX,2018,3.341195e+06,tonnes_gas_per_tonnes_fuel,Phoenix,789156.448948,2552038.605192,323.388171
2,US PHX,2020,2.274512e+06,tonnes_gas_per_tonnes_fuel,Phoenix,1056081.056728,1218431.109649,115.372878
4,US NYC,2016,3.379430e+06,tonnes_gas_per_tonnes_fuel,New York City,2425.695206,3377004.075996,139217.988634
5,US NYC,2016,2.290002e+06,tonnes_gas_per_tonnes_fuel,New York City,2425.695206,2287576.219249,94306.004057
7,NL RTM,2019,4.140259e+02,tonnes_gas_per_tonnes_fuel,Rotterdam,15207,-14792.97413,-97.277399
8,BR SSA,2015,4.883964e+05,tonnes_gas_per_tonnes_fuel,Salvador,485676,2720.384541,0.560123
9,BR SSA,2016,4.020770e+05,tonnes_gas_per_tonnes_fuel,Salvador,402170,-92.971837,-0.023118
10,BR SSA,2017,3.997116e+05,tonnes_gas_per_tonnes_fuel,Salvador,388280,11431.564744,2.944155
11,US LAX,2016,2.676924e+02,tonnes_gas_per_tonnes_fuel,Los Angeles,1174165.093573,-1173897.401151,-99.977201
